<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/%EA%B3%B5%EC%A1%B0_%EC%8B%9C%EC%8A%A4%ED%85%9C_%EC%B5%9C%EC%A0%81%ED%99%94_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_%ED%94%84%EB%A0%88%EC%9E%84%EC%9B%8C%ED%81%AC_(%EA%B0%9C%EB%8F%84%EC%9C%A8_%EB%AA%A8%EB%8D%B8_%EA%B3%A0%EB%8F%84%ED%99%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

# --- 1. 상수 정의 ---
# OAC 관련
NUM_OACS = 14
OAC_COIL_STAGES = ['preheat', 'precool', 'cool', 'reheat', 'humidify']
# OAC 코일 개도율 상하한 (%)
COIL_VALVE_MIN = 0
COIL_VALVE_MAX = 100

# 냉동기 관련
NUM_LT_CHILLERS = 8  # 저온 냉동기
NUM_HT_CHILLERS = 8  # 고온 냉동기
CHILLER_DP_SETPOINT = 1.5  # bar 또는 적절한 단위 (현업 설정값)
TARGET_CHILLER_LOAD_FACTOR = 0.55 # 55% (현업 설정값)
# 공기 질량 유량 (kg/s per OAC) - 가정치, 실제 데이터로 산출 필요
AIR_MASS_FLOW_PER_OAC_KG_S = 15.0

# 냉각탑 관련
NUM_COOLING_TOWERS = 9

# OAC 코일 출구 상태 예측 시 사용될 상수
CP_AIR_APPROX = 1.005 # 건공기 정압 비열 근사치 (kJ/kg.K)
K_RH_COOL_PER_DEG_C = 4.0 # 냉각 시 온도 1도 강하당 RH 증가율 근사치 (%)
K_RH_HEAT_PER_DEG_C = 4.0 # 가열 시 온도 1도 상승당 RH 감소율 근사치 (%)


# --- 2. 엔탈피 계산 함수 (단순화) ---
def calculate_enthalpy(temp_c, rel_humidity_percent):
    """
    온도(섭씨)와 상대습도(%)로 엔탈피(kJ/kg 건공기)를 근사적으로 계산합니다.
    매우 단순화된 버전입니다.
    """
    enthalpy = 1.005 * temp_c + (0.01 * rel_humidity_percent) * (2.5 * temp_c + 50)
    if temp_c < 5:
        enthalpy *= 0.9
    elif temp_c > 25:
        enthalpy *= 1.1
    return enthalpy


# --- 3. OAC 코일 개도율 예측 모듈 ---
class OACOptimizer:
    def __init__(self, num_oacs=NUM_OACS):
        self.num_oacs = num_oacs
        self.coil_properties = {
            'preheat': {'max_delta_h': 20, 'sensitivity': 1.0, 'valve_exponent': 0.9}, # kJ/kg, 낮은 수요에 약간 더 민감
            'precool': {'max_delta_h': 30, 'sensitivity': 1.0, 'valve_exponent': 1.0}, # kJ/kg, 선형
            'cool':    {'max_delta_h': 40, 'sensitivity': 1.0, 'valve_exponent': 1.1}, # kJ/kg, 낮은 수요에 약간 덜 민감
            'reheat':  {'max_delta_h': 15, 'sensitivity': 1.0, 'valve_exponent': 1.0}, # kJ/kg, 선형
            'humidify':{'max_delta_rh': 50, 'sensitivity': 1.5} # %RH, 가습은 습도차 기준, 민감도 조정
        }

    def _calculate_coil_output(self, stage_name, current_temp, current_rh, current_enthalpy,
                               target_sp_temp, target_sp_rh, manual_valve_override=None):
        """단일 코일 단계의 출력을 계산합니다 (개도율 모델 고도화)."""
        delta_h_actual = 0
        valve_opening = 0

        if stage_name == 'humidify': # 가습 코일
            rh_diff = target_sp_rh - current_rh
            if manual_valve_override is not None:
                valve_opening = np.clip(manual_valve_override, COIL_VALVE_MIN, COIL_VALVE_MAX)
                rh_change_estimated = (valve_opening / 100.0) * self.coil_properties[stage_name]['max_delta_rh'] * self.coil_properties[stage_name]['sensitivity']
                next_rh = np.clip(current_rh + rh_change_estimated, 0, 100)
                delta_h_actual = calculate_enthalpy(current_temp, next_rh) - current_enthalpy
            elif rh_diff > 0:
                valve_opening = np.clip((rh_diff / self.coil_properties[stage_name]['max_delta_rh']) * 100 * self.coil_properties[stage_name]['sensitivity'], COIL_VALVE_MIN, COIL_VALVE_MAX)
                # 민감도는 개도율에 반영되었으므로, 실제 RH 변화는 개도율에 따름
                rh_change_estimated = (valve_opening / 100.0) * self.coil_properties[stage_name]['max_delta_rh'] # 민감도 중복 적용 방지
                next_rh = np.clip(current_rh + rh_change_estimated, 0, 100)
                delta_h_actual = calculate_enthalpy(current_temp, next_rh) - current_enthalpy
            else:
                valve_opening = 0
                next_rh = current_rh

            next_temp = current_temp
            next_enthalpy = current_enthalpy + delta_h_actual

        else: # 온도 제어 코일 (예열, 예냉, 냉각, 승온)
            temp_diff = target_sp_temp - current_temp

            if temp_diff < 0:
                target_enthalpy = calculate_enthalpy(target_sp_temp, min(95, current_rh + abs(temp_diff) * 1.5)) # 습도 변화 더 고려
            else:
                target_enthalpy = calculate_enthalpy(target_sp_temp, max(10, current_rh - abs(temp_diff)*0.5)) # 가열시 RH 감소 고려

            required_delta_h = target_enthalpy - current_enthalpy

            # 밸브 개도율 결정
            if manual_valve_override is not None:
                valve_opening = np.clip(manual_valve_override, COIL_VALVE_MIN, COIL_VALVE_MAX)
            else:
                # 필요한 엔탈피 변화가 있고, 코일이 해당 방향으로 작동 가능한 경우
                is_heating_coil = stage_name in ['preheat', 'reheat']
                is_cooling_coil = stage_name in ['precool', 'cool']

                # 가열 코일은 required_delta_h > 0 일 때, 냉각 코일은 required_delta_h < 0 일 때 작동
                if (is_heating_coil and required_delta_h > 0) or \
                   (is_cooling_coil and required_delta_h < 0):

                    if np.isclose(self.coil_properties[stage_name]['max_delta_h'], 0):
                         valve_opening = 0 # max_delta_h가 0이면 개도율 0
                    else:
                        demand_ratio = abs(required_delta_h) / self.coil_properties[stage_name]['max_delta_h']
                        demand_ratio = np.clip(demand_ratio, 0, 1) # 수요 비율은 0과 1 사이

                        exponent = self.coil_properties[stage_name].get('valve_exponent', 1.0)
                        valve_opening_normalized = demand_ratio ** exponent
                        valve_opening = np.clip(valve_opening_normalized * 100, COIL_VALVE_MIN, COIL_VALVE_MAX)
                else:
                    valve_opening = 0 # 필요 없거나 반대 방향이면 개도율 0

            # 실제 엔탈피 변화량 계산 (밸브 개도율과 코일 특성 기반)
            potential_max_h_change_abs = self.coil_properties[stage_name]['max_delta_h']
            sensitivity = self.coil_properties[stage_name]['sensitivity']
            coil_type_sign = 0
            if stage_name in ['preheat', 'reheat']: coil_type_sign = 1  # 가열 코일
            elif stage_name in ['precool', 'cool']: coil_type_sign = -1 # 냉각 코일

            delta_h_actual = (valve_opening / 100.0) * (potential_max_h_change_abs * coil_type_sign) * sensitivity
            next_enthalpy = current_enthalpy + delta_h_actual

            # 출구 온도 및 습도 예측
            if np.isclose(valve_opening, 0) or np.isclose(delta_h_actual,0): # 밸브가 닫혔거나 엔탈피 변화가 없으면
                next_temp = current_temp
                next_rh = current_rh
            else:
                delta_temp_estimated = delta_h_actual / CP_AIR_APPROX
                next_temp = current_temp + delta_temp_estimated

                if delta_h_actual < 0: # 냉각 과정
                    rh_change = abs(delta_temp_estimated) * K_RH_COOL_PER_DEG_C
                    next_rh = min(100, current_rh + rh_change)
                elif delta_h_actual > 0: # 가열 과정
                    rh_change = abs(delta_temp_estimated) * K_RH_HEAT_PER_DEG_C
                    next_rh = max(5, current_rh - rh_change)
                else: # 엔탈피 변화 없으면 습도도 그대로
                    next_rh = current_rh
                next_rh = np.clip(next_rh, 5, 100) # RH 범위 제한
                next_temp = np.clip(next_temp, -20, 60) # 온도 범위 현실적으로 제한 (필요시 조정)

        return valve_opening, delta_h_actual, next_temp, next_rh, next_enthalpy

    def predict_oac_states(self, outdoor_temp, outdoor_rh, oac_setpoints_list, manual_overrides_list=None):
        all_oac_results = []
        total_precool_delta_h_sum = 0
        total_cool_delta_h_sum = 0

        for i in range(self.num_oacs):
            setpoints = oac_setpoints_list[i]
            current_oac_override_settings = None
            if manual_overrides_list and i < len(manual_overrides_list):
                current_oac_override_settings = manual_overrides_list[i]
            manual_overrides = current_oac_override_settings if current_oac_override_settings is not None else {}

            current_temp = outdoor_temp
            current_rh = outdoor_rh
            current_enthalpy = calculate_enthalpy(current_temp, current_rh)

            oac_coil_data = {'oac_id': f"OAC_{i+1}"}
            oac_precool_delta_h = 0
            oac_cool_delta_h = 0

            for stage in OAC_COIL_STAGES:
                sp_temp_key = f"{stage}_sp_temp"
                sp_rh_key = f"{stage}_sp_rh"
                valve_override_key = f"{stage}_valve_%"

                target_sp_temp = setpoints.get(sp_temp_key, current_temp)
                target_sp_rh = setpoints.get(sp_rh_key, current_rh)
                manual_valve = manual_overrides.get(valve_override_key, None)

                valve, delta_h, next_t, next_rh, next_h = self._calculate_coil_output(
                    stage, current_temp, current_rh, current_enthalpy,
                    target_sp_temp, target_sp_rh, manual_valve
                )

                oac_coil_data[f"{stage}_valve_%"] = round(valve, 2)
                oac_coil_data[f"{stage}_outlet_temp_c"] = round(next_t, 2)
                oac_coil_data[f"{stage}_outlet_rh_%"] = round(next_rh, 2)
                oac_coil_data[f"{stage}_delta_h_kj_kg"] = round(delta_h, 2)

                if stage == 'precool': oac_precool_delta_h = delta_h
                elif stage == 'cool': oac_cool_delta_h = delta_h

                current_temp, current_rh, current_enthalpy = next_t, next_rh, next_h

            all_oac_results.append(oac_coil_data)
            total_precool_delta_h_sum += abs(oac_precool_delta_h)
            total_cool_delta_h_sum += abs(oac_cool_delta_h)

        total_precool_load_kw = total_precool_delta_h_sum * AIR_MASS_FLOW_PER_OAC_KG_S
        total_cool_load_kw = total_cool_delta_h_sum * AIR_MASS_FLOW_PER_OAC_KG_S

        return all_oac_results, total_precool_load_kw, total_cool_load_kw


# --- 4. 냉동기 부하 및 가동대수 최적화 모듈 ---
class ChillerOptimizer:
    def __init__(self, num_lt_chillers=NUM_LT_CHILLERS, num_ht_chillers=NUM_HT_CHILLERS):
        self.num_lt_chillers = num_lt_chillers
        self.num_ht_chillers = num_ht_chillers
        self.lt_chiller_specs = sorted([
            (100, 3.5), (105, 3.6), (98, 3.4), (102, 3.55),
            (110, 3.7), (100, 3.3), (95, 3.2), (108, 3.65)
        ], key=lambda x: x[1], reverse=True)
        self.ht_chiller_specs = sorted([
            (80, 4.0), (85, 4.1), (78, 3.9), (82, 4.05),
            (90, 4.2), (80, 3.8), (75, 3.7), (88, 4.15)
        ], key=lambda x: x[1], reverse=True)

    def _optimize_single_chiller_type(self, total_load_kw, chiller_specs, num_chillers_available):
        chillers_on = 0
        current_total_capacity = 0
        running_chiller_indices_cop_ordered = []

        if total_load_kw <= 0:
            return 0, [], 0, 0, 0, 0, 0

        required_total_capacity_at_target_load = total_load_kw / TARGET_CHILLER_LOAD_FACTOR if TARGET_CHILLER_LOAD_FACTOR > 0 else total_load_kw

        for i in range(num_chillers_available):
            if current_total_capacity < required_total_capacity_at_target_load or chillers_on == 0:
                if chillers_on < num_chillers_available:
                    current_total_capacity += chiller_specs[i][0]
                    running_chiller_indices_cop_ordered.append(i)
                    chillers_on += 1
            else:
                break

        estimated_load_factor_percent = (total_load_kw / current_total_capacity) * 100 if current_total_capacity > 0 else 0
        main_dp = CHILLER_DP_SETPOINT if chillers_on > 0 else 0
        main_dp_valve = 0
        if chillers_on > 0 and current_total_capacity > 0:
            base_valve = estimated_load_factor_percent * 0.8
            valve_penalty = (chillers_on - 1) * 5
            main_dp_valve = np.clip(base_valve - valve_penalty , 10, 95)
        sub_dp_valve = main_dp_valve

        return (chillers_on, running_chiller_indices_cop_ordered,
                round(current_total_capacity,2), round(estimated_load_factor_percent,2),
                main_dp, round(main_dp_valve,2), round(sub_dp_valve,2))

    def optimize_chiller_systems(self, total_precool_load_kw, total_cool_load_kw):
        lt_on, lt_idx, lt_cap, lt_load_f, lt_dp, lt_main_v, lt_sub_v = \
            self._optimize_single_chiller_type(total_cool_load_kw, self.lt_chiller_specs, self.num_lt_chillers)
        ht_on, ht_idx, ht_cap, ht_load_f, ht_dp, ht_main_v, ht_sub_v = \
            self._optimize_single_chiller_type(total_precool_load_kw, self.ht_chiller_specs, self.num_ht_chillers)

        lt_results = {
            "type": "Low Temperature Chillers", "num_chillers_on": lt_on,
            "running_chiller_indices_by_cop_rank": lt_idx, "total_load_kW": round(total_cool_load_kw,2),
            "current_operating_capacity_kW": lt_cap, "estimated_avg_load_factor_%": lt_load_f,
            "main_dp_set_kpa_or_bar": lt_dp, "main_dp_valve_%": lt_main_v, "sub_dp_valve_%": lt_sub_v
        }
        ht_results = {
            "type": "High Temperature Chillers", "num_chillers_on": ht_on,
            "running_chiller_indices_by_cop_rank": ht_idx, "total_load_kW": round(total_precool_load_kw,2),
            "current_operating_capacity_kW": ht_cap, "estimated_avg_load_factor_%": ht_load_f,
            "main_dp_set_kpa_or_bar": ht_dp, "main_dp_valve_%": ht_main_v, "sub_dp_valve_%": ht_sub_v
        }
        return {"low_temp_chillers_results": lt_results, "high_temp_chillers_results": ht_results}

# --- 5. 냉각탑 가동대수 최적화 모듈 ---
class CoolingTowerOptimizer:
    def __init__(self, num_towers=NUM_COOLING_TOWERS):
        self.num_towers = num_towers
        self.tower_specs = sorted([
            (200, 0.95), (220, 0.96), (200, 0.94), (250, 0.98),
            (210, 0.93), (200, 0.92), (230, 0.97), (240, 0.95), (260, 0.99)
        ], key=lambda x: x[1], reverse=True)

    def optimize_tower_operation(self, chiller_results, lt_chiller_specs_sorted_by_cop, ht_chiller_specs_sorted_by_cop):
        lt_ch_res = chiller_results['low_temp_chillers_results']
        ht_ch_res = chiller_results['high_temp_chillers_results']
        total_heat_rejection_load_kw = 0

        if lt_ch_res['num_chillers_on'] > 0 and lt_ch_res['total_load_kW'] > 0:
            running_lt_cops = [lt_chiller_specs_sorted_by_cop[i][1] for i in lt_ch_res['running_chiller_indices_by_cop_rank']]
            avg_lt_cop = np.mean(running_lt_cops) if running_lt_cops else 1
            total_heat_rejection_load_kw += lt_ch_res['total_load_kW'] * (1 + 1/avg_lt_cop if avg_lt_cop > 0 else 1)

        if ht_ch_res['num_chillers_on'] > 0 and ht_ch_res['total_load_kW'] > 0:
            running_ht_cops = [ht_chiller_specs_sorted_by_cop[i][1] for i in ht_ch_res['running_chiller_indices_by_cop_rank']]
            avg_ht_cop = np.mean(running_ht_cops) if running_ht_cops else 1
            total_heat_rejection_load_kw += ht_ch_res['total_load_kW'] * (1 + 1/avg_ht_cop if avg_ht_cop > 0 else 1)

        towers_on = 0
        current_tower_capacity = 0
        running_tower_indices_by_efficiency_rank = []

        if total_heat_rejection_load_kw > 0:
            for i in range(self.num_towers):
                if current_tower_capacity < total_heat_rejection_load_kw:
                    if towers_on < self.num_towers:
                        current_tower_capacity += self.tower_specs[i][0]
                        running_tower_indices_by_efficiency_rank.append(i)
                        towers_on += 1
                else:
                    break
            if towers_on == 0 :
                current_tower_capacity = self.tower_specs[0][0]
                running_tower_indices_by_efficiency_rank.append(0)
                towers_on = 1

        return {
            "num_towers_on": towers_on,
            "running_tower_indices_by_efficiency_rank": running_tower_indices_by_efficiency_rank,
            "total_heat_rejection_load_kW": round(total_heat_rejection_load_kw,2),
            "current_total_operating_capacity_kW": round(current_tower_capacity,2)
        }

# --- 6. 메인 실행 로직 및 더미 데이터 ---
if __name__ == "__main__":
    outdoor_air_temp_input = 28.0
    outdoor_air_rh_input = 65.0

    single_oac_sp = {
        'preheat_sp_temp': 15.0, 'precool_sp_temp': 22.0,
        'cool_sp_temp': 18.0, 'reheat_sp_temp': 20.0,
        'humidify_sp_rh': 50.0
    }
    oac_target_setpoints_input = [single_oac_sp.copy() for _ in range(NUM_OACS)]

    oac_manual_overrides_input = [None] * NUM_OACS
    if NUM_OACS >= 1: oac_manual_overrides_input[0] = {'precool_valve_%': 70.0}
    if NUM_OACS >= 2: oac_manual_overrides_input[1] = {'cool_valve_%': 60.0, 'reheat_valve_%': 10.0}
    if NUM_OACS >= 3: oac_manual_overrides_input[2] = None

    print(f"--- 입력 조건 ---")
    print(f"외기 온도: {outdoor_air_temp_input}°C, 외기 습도: {outdoor_air_rh_input}%")
    if NUM_OACS > 0:
        print(f"OAC Setpoints (OAC_1 예시): {oac_target_setpoints_input[0]}")
        if oac_manual_overrides_input[0]: print(f"OAC_1 수동 개도율 설정: {oac_manual_overrides_input[0]}")
    if NUM_OACS > 1 and oac_manual_overrides_input[1]: print(f"OAC_2 수동 개도율 설정: {oac_manual_overrides_input[1]}")
    if NUM_OACS > 2 and oac_manual_overrides_input[2] is None: print(f"OAC_3 수동 개도율 설정: 없음")
    print("-" * 30)

    oac_model = OACOptimizer(num_oacs=NUM_OACS)
    chiller_model = ChillerOptimizer()
    lt_chiller_specs_for_tower = chiller_model.lt_chiller_specs
    ht_chiller_specs_for_tower = chiller_model.ht_chiller_specs
    tower_model = CoolingTowerOptimizer()

    oac_predictions, total_ht_chiller_load, total_lt_chiller_load = \
        oac_model.predict_oac_states(
            outdoor_air_temp_input, outdoor_air_rh_input,
            oac_target_setpoints_input, oac_manual_overrides_input
        )

    chiller_system_results = chiller_model.optimize_chiller_systems(
        total_ht_chiller_load, total_lt_chiller_load
    )

    cooling_tower_results = tower_model.optimize_tower_operation(
        chiller_system_results, lt_chiller_specs_for_tower, ht_chiller_specs_for_tower
    )

    print("\n--- OAC 코일별 개도율 예측 결과 (14대 OAC) ---")
    # 모든 OAC 결과 출력 (생략 없음)
    for i in range(NUM_OACS):
        print(f"\n{oac_predictions[i]['oac_id']} Predictions:")
        for stage in OAC_COIL_STAGES:
            print(f"  {stage.capitalize():<8} Valve: {oac_predictions[i].get(f'{stage}_valve_%', 'N/A'):>5}% "
                  f"| Outlet Temp: {oac_predictions[i].get(f'{stage}_outlet_temp_c', 'N/A'):>5}°C "
                  f"| Outlet RH: {oac_predictions[i].get(f'{stage}_outlet_rh_%', 'N/A'):>5}% "
                  f"| Delta H: {oac_predictions[i].get(f'{stage}_delta_h_kj_kg', 'N/A'):>6} kJ/kg")

    print("\n\n--- 저온 냉동기 예측 결과 ---")
    lt_res = chiller_system_results['low_temp_chillers_results']
    print(f"  적정 가동 대수: {lt_res['num_chillers_on']} / {NUM_LT_CHILLERS} 대")
    print(f"  가동 냉동기 Index (COP 높은 순): {lt_res['running_chiller_indices_by_cop_rank']}")
    print(f"  총 부하: {lt_res['total_load_kW']} kW")
    print(f"  현재 가동 용량: {lt_res['current_operating_capacity_kW']} kW")
    print(f"  예상 평균 부하율: {lt_res['estimated_avg_load_factor_%']}% (목표 평균: {TARGET_CHILLER_LOAD_FACTOR*100}%)")
    print(f"  메인 차압: {lt_res['main_dp_set_kpa_or_bar']} (단위 확인 필요, 설정 목표: {CHILLER_DP_SETPOINT})")
    print(f"  메인 차압 밸브 개도율: {lt_res['main_dp_valve_%']}%")
    print(f"  서브 차압 밸브 개도율: {lt_res['sub_dp_valve_%']}%")

    print("\n--- 고온 냉동기 예측 결과 ---")
    ht_res = chiller_system_results['high_temp_chillers_results']
    print(f"  적정 가동 대수: {ht_res['num_chillers_on']} / {NUM_HT_CHILLERS} 대")
    print(f"  가동 냉동기 Index (COP 높은 순): {ht_res['running_chiller_indices_by_cop_rank']}")
    print(f"  총 부하: {ht_res['total_load_kW']} kW")
    print(f"  현재 가동 용량: {ht_res['current_operating_capacity_kW']} kW")
    print(f"  예상 평균 부하율: {ht_res['estimated_avg_load_factor_%']}% (목표 평균: {TARGET_CHILLER_LOAD_FACTOR*100}%)")
    print(f"  메인 차압: {ht_res['main_dp_set_kpa_or_bar']} (단위 확인 필요, 설정 목표: {CHILLER_DP_SETPOINT})")
    print(f"  메인 차압 밸브 개도율: {ht_res['main_dp_valve_%']}%")
    print(f"  서브 차압 밸브 개도율: {ht_res['sub_dp_valve_%']}%")

    print("\n--- 냉각탑 예측 결과 ---")
    print(f"  적정 가동 대수: {cooling_tower_results['num_towers_on']} / {NUM_COOLING_TOWERS} 대")
    print(f"  가동 냉각탑 Index (효율 높은 순): {cooling_tower_results['running_tower_indices_by_efficiency_rank']}")
    print(f"  총 열 제거 부하 (냉동기 부하 + 추정 소비전력): {cooling_tower_results['total_heat_rejection_load_kW']} kW")
    print(f"  현재 가동 냉각탑 총 용량: {cooling_tower_results['current_total_operating_capacity_kW']} kW")

--- 입력 조건 ---
외기 온도: 28.0°C, 외기 습도: 65.0%
OAC Setpoints (OAC_1 예시): {'preheat_sp_temp': 15.0, 'precool_sp_temp': 22.0, 'cool_sp_temp': 18.0, 'reheat_sp_temp': 20.0, 'humidify_sp_rh': 50.0}
OAC_1 수동 개도율 설정: {'precool_valve_%': 70.0}
OAC_2 수동 개도율 설정: {'cool_valve_%': 60.0, 'reheat_valve_%': 10.0}
OAC_3 수동 개도율 설정: 없음
------------------------------

--- OAC 코일별 개도율 예측 결과 (14대 OAC) ---

OAC_1 Predictions:
  Preheat  Valve:     0% | Outlet Temp:  28.0°C | Outlet RH:  65.0% | Delta H:    0.0 kJ/kg
  Precool  Valve:  70.0% | Outlet Temp:   7.1°C | Outlet RH:   100% | Delta H:  -21.0 kJ/kg
  Cool     Valve:     0% | Outlet Temp:   7.1°C | Outlet RH:   100% | Delta H:   -0.0 kJ/kg
  Reheat   Valve: 100.0% | Outlet Temp: 22.03°C | Outlet RH:  40.3% | Delta H:   15.0 kJ/kg
  Humidify Valve:  29.1% | Outlet Temp: 22.03°C | Outlet RH: 54.85% | Delta H: -30.98 kJ/kg

OAC_2 Predictions:
  Preheat  Valve:     0% | Outlet Temp:  28.0°C | Outlet RH:  65.0% | Delta H:    0.0 kJ/kg
  Precool  Valve: 56.48%